In [1]:
import cv2
import os
import numpy as np
import glob
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold as kf
from PIL import Image

In [2]:
# Load Image which saved in .npz files
dataset_path = "utils//dataset.npz"
data = np.load(dataset_path)

x = data['x']
y = data['y']

In [3]:
# Cross Validation
kf_model = kf(n_splits=5, random_state=10, shuffle=True)
for train_idx, test_idx in kf_model.split(x, y):
    x_train, y_train = x[train_idx] , y[train_idx]
    x_test, y_test = x[test_idx] , y[test_idx]

In [4]:
# Calculate ratio each classes
# Gather data state for further augmentation
from collections import Counter

augmentation_target = 2000
num_classes = 0
ns = []

counter = Counter(y_train)
for k, v in counter.items():
    per= v / len(y) * 100
    ns.append(v)
    num_classes = num_classes + 1
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=0, n=96 (4.548%)
Class=1, n=171 (8.100%)
Class=2, n=159 (7.532%)
Class=3, n=77 (3.648%)
Class=4, n=123 (5.827%)
Class=5, n=77 (3.648%)
Class=6, n=118 (5.590%)
Class=7, n=76 (3.600%)
Class=8, n=154 (7.295%)
Class=9, n=80 (3.790%)
Class=10, n=132 (6.253%)
Class=11, n=139 (6.585%)
Class=12, n=208 (9.853%)
Class=13, n=79 (3.742%)


In [5]:
num_classes

14

In [6]:
# Gather difference between each classes ratio for augmentation combination
required_combination = []

for idx, n in enumerate(ns):
    required_combination.append(augmentation_target // n)
        
required_combination

[20, 11, 12, 25, 16, 25, 16, 26, 12, 25, 15, 14, 9, 25]

<b>Augmentation Combination</b>

First Group

0. None
1. Horizontal Flip
2. Gausian Blur
3. Gausian Noise
4. Speckle Noise

Second Group

0. None
1. Contrast
2. Brightness
3. Hue
4. Saturation
5. Exposure

Possible Combination is 25

In [7]:
import random

def generate_random_combination(n_combination=2):
    combination = []
    
    itr = 0
    # Looping to generate n combination
    # Only possible for 11 combination, if 11 combination has been fulfilled, loop may go infinity
    while itr < n_combination:
        # Generate within range 0f 0-3 and 0-2
        temp = [random.randrange(0, 5), random.randrange(0, 6)]
        
        # Avoid combination [0, 0]
        if temp == [0, 0]:
            continue
        
        # Assign for first iteration
        if itr == 0:
            combination.append(temp)
            itr += 1
        
        # Assign for second and later iteration
        if itr != 0:      
            # Gather Information upwards/backward of array
            isTheSame = False
            for data in combination:
                if data == temp:
                    isTheSame = True
                    break
                    
            # if no similiarity to backward array temp succesfully added
            if isTheSame == False:
                combination.append(temp)
                itr += 1
            else:
                continue

    return np.asarray(combination)

In [8]:
# Augmentation Action
def brightness(img, brightness=255):    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        buf = cv2.addWeighted(img, alpha_b, img, 0, gamma_b)
    else:
        buf = img.copy()
        
    return buf

def contrast(img, alpha=1.5, beta=20):
    return cv2.addWeighted(img, alpha, np.zeros(img.shape, img.dtype), 0, beta)

def gaussian_noise(img, intensity=0.5):
    gauss = np.random.normal(0,intensity,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
    noise = np.add(img, gauss)
    
    return noise

def speckle_noise(img, intensity=0.5):
    gauss = np.random.normal(0,intensity,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
    noise = img + img * gauss
    
    return noise

def hue_shift(image, degree_shift=180):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    shift_h = (h + 90) % degree_shift
    shift_hsv = cv2.merge([shift_h, s, v])
    
    return cv2.cvtColor(shift_hsv, cv2.COLOR_HSV2RGB)

def saturation(image, saturation=10, scale=1):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    apply_s = (s * scale) + saturation
    apply_hsv = cv2.merge([h, apply_s, v])
    
    return cv2.cvtColor(apply_hsv, cv2.COLOR_HSV2RGB)

def exposure_fusion(img):
    exposure = [
        brightness(img, brightness=-50),
        brightness(img, brightness=-30),
        brightness(img, brightness=30),
        brightness(img, brightness=50)
    ]
    
    # Align image
    alligned = cv2.createAlignMTB()
    alligned.process(exposure, exposure)
    
    # Merge image
    merged = cv2.createMergeMertens()
    exposureFusion = merged.process(exposure)

    #info = np.iinfo(exposureFusion.dtype)
    #data = data.astype(np.float32) / info.max
    data = 255 * exposureFusion
    img = data.astype(np.uint8)
    
    return img

In [9]:
# Augmentation Function Handler
def augmentation_handler(img, path, combination=[1, 1]):
    def first_group(img, num):
        if num == 0:
            return img
        if num == 1:
            return cv2.flip(img, 1)
        if num == 2:
            return cv2.GaussianBlur(img,(3,3),cv2.BORDER_DEFAULT)
        if num == 3:
            return gaussian_noise(img, intensity=0.5)
        if num == 4:
            return speckle_noise(img, intensity=0.5)
    def second_group(img, num):
        if num == 0:
            return img
        if num == 1:
            return contrast(img, alpha=1.1, beta=15)
        if num == 2:
            return brightness(img, brightness=50)
        if num == 3:
            return hue_shift(img, degree_shift=120)
        if num == 4:
            return saturation(img, saturation=15, scale=2)
        if num == 5:
            return exposure_fusion(img)
        
    first_result  = first_group(img, combination[0])
    final_result  = second_group(first_result, combination[1])
    
    return final_result

In [10]:
def min_max(img):
    image = img.astype('float32')
    image /= 255.0
    
    return image

In [11]:
# Generate Possible All Possible Combination and avoiding [0, 0]
imwrite_path = 'Dataset_augmented/train'
current_img = 0

for idx in range(num_classes):
    # Generate Random Combination for augmentation Each Classes
    combination = generate_random_combination(required_combination[idx])
    
    # Eliminate empty combination(Eg. Class which dont need any augmentation)
    if len(combination) == 0:
        continue
        
    # loop the Combination array
    current_class = idx
    for idx_comb, combinated in enumerate(combination):
        # Loop Through the image
        for idx_img, img in enumerate(x_train):
            if y[idx_img] == current_class:
                # Check for the first Augmentation
                path = os.path.join(imwrite_path, str(current_img + 1) + "_augmented_train_" + str(current_class) + ".png")
                
                final_img = augmentation_handler(img, path, combinated)
                
                cv2.imwrite(path, final_img)
                current_img = current_img + 1

In [12]:
for idx, img in enumerate(x_train):
    path = os.path.join(imwrite_path, str(idx + 1) + "_original_train_" + str(y_train[idx]) + ".png")
    cv2.imwrite(path, img)

In [13]:
imwrite_path = 'Dataset_augmented/test'
for idx, img in enumerate(x_test):
    path = os.path.join(imwrite_path, str(idx + 1) + "_original_train_" + str(y_test[idx]) + ".png")
    cv2.imwrite(path, img)